<a href="https://colab.research.google.com/github/joannedonohue/CryptoClustering/blob/main/Data_Axel_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [266]:
# Upload CSV
from google.colab import files
import pandas as pd

uploaded = files.upload()  # This opens a file picker in the UI

# Get the uploaded file name
filename = next(iter(uploaded))



Saving Detail2025080415595731.csv to Detail2025080415595731.csv


In [267]:
# Load CSV into DataFrame
df = pd.read_csv(filename)

df.head()


,Company Name,Parent Company Name,Executive First Name,Executive Last Name,Professional Title,Executive Title,Executive Gender,Address,City,State,...,Franchise/Specialty Code 5.0,Franchise/Specialty Code 5.1,Franchise/Specialty Code 6.0,Affiliated Records,Franchise/Specialty Code 6.1,Affiliated Locations,Federal Contractor,Census Block Group,Legal Name,Record Type
0,ABM Digital,NaN,Jerry,Martinez,NaN,Owner,Male,8420 W Myrtle Ave,Glendale,AZ,...,NaN,NaN,NaN,0,NaN,0,No,1,NaN,Unverified
1,Abs Inc,NaN,Dave,Willis,NaN,Owner,Male,7729 E Greenway Rd # 200,Scottsdale,AZ,...,NaN,NaN,NaN,0,NaN,0,No,1,ABS INC,Verified
2,All Copy Products,ALL COPY PRODUCTS,Scott,Baker,NaN,Manager,Male,1407 W Drivers Way,Tempe,AZ,...,NaN,NaN,NaN,0,NaN,0,No,2,ALL COPY PRODUCTS INC,Unverified
3,All Copy Products,ALL COPY PRODUCTS,NaN,NaN,NaN,NaN,NaN,415 S 48th St,Tempe,AZ,...,NaN,NaN,NaN,0,NaN,0,No,1,NaN,Verified
4,Alpha Graphics,BLACKSTREET CAPITAL MGMT LLC,NaN,NaN,NaN,NaN,NaN,6680 W Bell Rd # C,Glendale,AZ,...,NaN,NaN,NaN,0,NaN,0,No,3,NaN,Unverified


In [268]:
# Clean phone number formatting

import re

def clean_phone_number(val):
    if pd.isna(val):
        return None
    # Remove everything except digits
    digits = re.sub(r"\D", "", str(val))
    # Keep only if it's 10 digits
    if len(digits) == 10:
        return f"+1{digits}"
    elif len(digits) == 11 and digits.startswith("1"):
        return f"+{digits}"  # Already includes country code
    else:
        return None  # Invalid or unexpected format

# Columns to clean
phone_columns = [
    "Phone Number Combined",
    "Fax Number Combined",
    "Toll Free Number Combined"
]

# Apply cleaning function to each column
for col in phone_columns:
    if col in df.columns:
        df[col] = df[col].apply(clean_phone_number)

In [269]:
# Combine First and Last Name throughout

# Combine first and last names for the Primary Executive
df["Executive Name Combined"] = df["Executive First Name"].fillna("").str.strip() + " " + df["Executive Last Name"].fillna("").str.strip()

# Clean up extra spaces
df["Executive Name Combined"] = df["Executive Name Combined"].str.strip().replace(r"\s+", " ", regex=True)


# Create full name column Exec 1, 2, 3
df["Executive Name Combined 1"] = df["Executive First Name 1"].fillna("").str.strip() + " " + df["Executive Last Name 1"].fillna("").str.strip()
df["Executive Name Combined 2"] = df["Executive First Name 2"].fillna("").str.strip() + " " + df["Executive Last Name 2"].fillna("").str.strip()
df["Executive Name Combined 3"] = df["Executive First Name 3"].fillna("").str.strip() + " " + df["Executive Last Name 3"].fillna("").str.strip()

# Clean up any double spaces if both names aren't present
df["Executive Name Combined 1"] = df["Executive Name Combined 1"].str.strip().replace(r"\s+", " ", regex=True)
df["Executive Name Combined 2"] = df["Executive Name Combined 2"].str.strip().replace(r"\s+", " ", regex=True)
df["Executive Name Combined 3"] = df["Executive Name Combined 3"].str.strip().replace(r"\s+", " ", regex=True)

In [270]:
# PO Box Combined

# Fill empty fields with empty strings and strip whitespace
df["PO Box Combined"] = (
    df["Mailing Address"].fillna("").str.strip()
    + ", " + df["Mailing City"].fillna("").str.strip()
    + ", " + df["Mailing State"].fillna("").str.strip()
    + " " + df["Mailing Zip Code"].fillna("").astype(str).str.strip()
)

# Clean up edge cases: remove extra commas or spaces if some fields are missing
df["PO Box Combined"] = df["PO Box Combined"].str.replace(r",\s*,", ",", regex=True)
df["PO Box Combined"] = df["PO Box Combined"].str.replace(r",\s*$", "", regex=True)
df["PO Box Combined"] = df["PO Box Combined"].str.strip()

In [271]:
# Step 3: Define the list of columns to keep
columns_to_keep = [
    "Company Name", "Parent Company Name", "Executive Name Combined", "Executive First Name", "Executive Last Name",
    "Executive Title", "Address", "City", "State", "ZIP Code", "Metro Area",
    "Phone Number Combined", "Fax Number Combined", "Toll Free Number Combined", "Website",
    "Company Description", "Primary SIC Code", "Primary SIC Description", "SIC Code 1", "SIC Code 1 Description",
    "Primary NAICS", "Primary NAICS Description", "NAICS 1", "NAICS 1 Description", "Location Employee Size Range",
    "Location Employee Size Actual", "Location Sales Volume Range", "Location Sales Volume Actual",
    "Corporate Employee Size Range", "Corporate Employee Size Actual", "Corporate Sales Volume Range",
    "Corporate Sales Volume Actual", "Type of Business", "Location Type", "Year Established", "Square Footage",
    "Credit Score Alpha", "Own or Lease", "Executive Name Combined 1", "Executive First Name 1", "Executive Last Name 1", "Executive Title 1",
    "Executive Name Combined 2", "Executive First Name 2", "Executive Last Name 2", "Executive Title 2", "Executive Name Combined 3", "Executive First Name 3",
    "Executive Last Name 3", "Executive Title 3", "PO Box Combined", "Mailing Address", "Mailing City", "Mailing State",
    "Mailing Zip Code", "Twitter", "Linked-In", "Facebook", "Federal Contractor"
]



In [272]:
# Step 4: Filter DataFrame to only include desired columns (if they exist in the file)
filtered_df = df[[col for col in columns_to_keep if col in df.columns]]



In [273]:
# Step 5: Display the first few rows
filtered_df.head()

,Company Name,Parent Company Name,Executive Name Combined,Executive First Name,Executive Last Name,Executive Title,Address,City,State,ZIP Code,...,Executive Title 3,PO Box Combined,Mailing Address,Mailing City,Mailing State,Mailing Zip Code,Twitter,Linked-In,Facebook,Federal Contractor
0,ABM Digital,NaN,Jerry Martinez,Jerry,Martinez,Owner,8420 W Myrtle Ave,Glendale,AZ,85305,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
1,Abs Inc,NaN,Dave Willis,Dave,Willis,Owner,7729 E Greenway Rd # 200,Scottsdale,AZ,85260,...,President,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
2,All Copy Products,ALL COPY PRODUCTS,Scott Baker,Scott,Baker,Manager,1407 W Drivers Way,Tempe,AZ,85284,...,Manager,,NaN,NaN,NaN,NaN,http://twitter.com/allcopy,http://www.linkedin.com/company/all-copy-products,NaN,No
3,All Copy Products,ALL COPY PRODUCTS,,NaN,NaN,NaN,415 S 48th St,Tempe,AZ,85281,...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
4,Alpha Graphics,BLACKSTREET CAPITAL MGMT LLC,,NaN,NaN,NaN,6680 W Bell Rd # C,Glendale,AZ,85308,...,NaN,,NaN,NaN,NaN,NaN,http://twitter.com/ag_oly,http://www.linkedin.com/pub/christopher-iacuzz...,http://www.facebook.com/alphagraphics,No


In [274]:
# Step 7: Save to CSV and offer download
output_filename = "filtered_output.csv"
filtered_df.to_csv(output_filename, index=False)
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>